In [21]:
import numpy as np 
import pandas as pd

In [22]:
df= pd.read_csv('../static/capes.csv')
df.head()

,Unnamed: 0,Instructor,Course,Quarter,Total Enrolled in Course,Total CAPEs Given,Percentage Recommended Class,Percentage Recommended Professor,StudyHoursperWeek,Average Grade Expected,Average Grade Received,Evalulation URL,CourseNum,dept
0,0,Butler Elizabeth Annette,AAS 10 - Intro/African-American Studies (A),SP23,66,48,93.5%,100.0%,2.80,A- (3.84),B+ (3.67),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,AAS 10,AAS
1,1,Butler Elizabeth Annette,AAS 170 - Legacies of Research (A),SP23,20,7,100.0%,100.0%,2.50,A- (3.86),A- (3.92),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,AAS 170,AAS
2,2,Jones Ian William Nasser,ANAR 111 - Foundations of Archaeology (A),SP23,16,3,100.0%,100.0%,3.83,B+ (3.67),NaN,https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,ANAR 111,ANAR
3,3,Shtienberg Gilad,ANAR 115 - Coastal Geomorphology/Environ (A),SP23,26,6,100.0%,83.3%,3.83,B+ (3.50),B (3.07),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,ANAR 115,ANAR
4,4,Braswell Geoffrey E.,ANAR 155 - Stdy Abrd: Ancient Mesoamerica (A),SP23,22,9,100.0%,100.0%,5.17,A (4.00),A (4.00),https://cape.ucsd.edu/CAPEReport.aspx?sectioni...,ANAR 155,ANAR


In [23]:
# Remove the "Unnamed: 0" column
df.drop(columns=['Unnamed: 0'], inplace=True)

# Convert percentage columns to numeric by removing '%' and converting to float
df['Percentage Recommended Class'] = df['Percentage Recommended Class'].str.rstrip('%').astype('float')
df['Percentage Recommended Professor'] = df['Percentage Recommended Professor'].str.rstrip('%').astype('float')

# Extract numeric grades from 'Average Grade Expected' and 'Average Grade Received'
df['Average Grade Expected'] = df['Average Grade Expected'].str.extract(r"\((.*)\)").astype('float')
df['Average Grade Received'] = df['Average Grade Received'].str.extract(r"\((.*)\)").astype('float')

# Create 'Year' and 'Quarter' from the 'Quarter' column
df['Year'] = '20' + df['Quarter'].str[2:]
df['Quarter'] = df['Quarter'].str[:2].replace({'SP': 'Spring', 'FA': 'Fall', 'WI': 'Winter'})
df['Quarter'] = df['Quarter'].replace(['S3', 'S2', 'S1', 'SU'], 'Spring')


# Rename columns to remove spaces
df.columns = df.columns.str.replace(' ', '')

# Check for NaN values
nan_counts = df.isna().sum()

# Fill NaN values for 'AverageGradeExpected' and 'AverageGradeReceived' with the mean of their 'CourseNum' group
course_means = df.groupby('CourseNum').transform('mean')
df['AverageGradeExpected'] = df['AverageGradeExpected'].fillna(course_means['AverageGradeExpected'])
df['AverageGradeReceived'] = df['AverageGradeReceived'].fillna(course_means['AverageGradeReceived'])

# Fill remaining NaNs with 0 if no mean value is available for the group
df['AverageGradeExpected'].fillna(0, inplace=True)
df['AverageGradeReceived'].fillna(0, inplace=True)


/var/folders/bj/qrdzcn1x2yj47667bynywzy80000gn/T/ipykernel_34246/2291781661.py:25: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  course_means = df.groupby('CourseNum').transform('mean')


In [24]:
df['Quarter'].unique()

array(['Spring', 'Winter', 'Fall'], dtype=object)

In [26]:
grouped_df = df.groupby('CourseNum').mean()

# Reset index to make 'CourseNum' a column again
grouped_df.reset_index(inplace=True)

/var/folders/bj/qrdzcn1x2yj47667bynywzy80000gn/T/ipykernel_34246/1288910322.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_df = df.groupby('CourseNum').mean()


In [30]:
grouped_df['dept'] = grouped_df['CourseNum'].str.split().str[0] 

In [33]:
df.dtypes

Instructor                         object
Course                             object
Quarter                            object
TotalEnrolledinCourse               int64
TotalCAPEsGiven                     int64
PercentageRecommendedClass        float64
PercentageRecommendedProfessor    float64
StudyHoursperWeek                 float64
AverageGradeExpected              float64
AverageGradeReceived              float64
EvalulationURL                     object
CourseNum                          object
dept                               object
Year                               object
dtype: object

In [36]:
grouped_df.to_csv('capes_grouped.csv')

In [9]:
grade_mapping = {
    'A+': 4.0,
    'A': 4.0,
    'A-': 3.7,
    'B+': 3.3,
    'B': 3.0,
    'B-': 2.7,
    'C+': 2.3,
    'C': 2.0,
    'C-': 1.7
}

In [35]:
grouped_df

,CourseNum,TotalEnrolledinCourse,TotalCAPEsGiven,PercentageRecommendedClass,PercentageRecommendedProfessor,StudyHoursperWeek,AverageGradeExpected,AverageGradeReceived,dept
0,AAPI 87,10.000000,3.000000,100.000000,100.000000,0.500000,4.000000,0.000000,AAPI
1,AAS 10,59.454545,34.454545,86.472727,85.172727,3.933636,3.670909,3.676364,AAS
2,AAS 11,65.000000,46.000000,93.500000,93.300000,4.150000,3.840000,3.710000,AAS
3,AAS 170,25.500000,8.000000,94.450000,94.450000,3.055000,3.875000,3.800000,AAS
4,AAS 190,28.750000,10.500000,100.000000,100.000000,2.097500,3.965000,4.000000,AAS
...,...,...,...,...,...,...,...,...,...
4399,WARR 87,12.117647,8.411765,92.188235,91.229412,1.391176,3.867000,0.000000,WARR
4400,WCWP 100,20.029586,12.727811,86.640828,96.340237,7.145207,3.463432,3.478148,WCWP
4401,WCWP 10A,99.260870,58.504348,75.310435,88.419130,6.975478,3.283565,3.323778,WCWP
4402,WCWP 10B,75.124138,37.393103,76.665517,90.519310,7.112690,3.343862,3.428542,WCWP
